# Importing a sentiment analysis model

In [4]:
from transformers import pipeline

# Load multilingual sentiment analysis model
sentiment_pipeline = pipeline('sentiment-analysis', model='cardiffnlp/twitter-xlm-roberta-base-sentiment')

In [11]:
# Provided conversation
conversation_text = """
Recovery Agent (RA): नमस्ते श्री कुमार, मैं एक्स वाई जेड फाइनेंस से बोल रहा हूं। आपके लोन के बारे में बात करनी थी।
Borrower (B): हां, बोलिए। क्या बात है?
RA: सर, आपका पिछले महीने का EMI अभी तक नहीं आया है। क्या कोई समस्या है?
B: हां, थोड़ी दिक्कत है। मेरी नौकरी चली गई है और मैं नया काम ढूंढ रहा हूं।
RA: ओह, यह तो बुरा हुआ। लेकिन सर, आपको समझना होगा कि लोन का भुगतान समय पर करना बहुत जरूरी है।
B: मैं समझता हूं, लेकिन अभी मेरे पास पैसे नहीं हैं। क्या कुछ समय मिल सकता है?
RA: हम समझते हैं आपकी स्थिति। क्या आप अगले हफ्ते तक कुछ भुगतान कर सकते हैं?
B: मैं कोशिश करूंगा, लेकिन पूरा EMI नहीं दे पाऊंगा। क्या आधा भुगतान चलेगा?
RA: ठीक है, आधा भुगतान अगले हफ्ते तक कर दीजिए। बाकी का क्या प्लान है आपका?
B: मुझे उम्मीद है कि अगले महीने तक मुझे नया काम मिल जाएगा। तब मैं बाकी बकाया चुका दूंगा।
RA: ठीक है। तो हम ऐसा करते हैं- आप अगले हफ्ते तक आधा EMI जमा कर दीजिए, और अगले महीने के 15 तारीख तक बाकी का भुगतान कर दीजिए। क्या यह आपको स्वीकार है?
B: हां, यह ठीक रहेगा। मैं इस प्लान का पालन करने की पूरी कोशिश करूंगा।
RA: बहुत अच्छा। मैं आपको एक SMS भेज रहा हूं जिसमें भुगतान की डिटेल्स होंगी। कृपया इसका पालन करें और समय पर भुगतान करें।
B: ठीक है, धन्यवाद आपके समझने के लिए।
RA: आपका स्वागत है। अगर कोई और सवाल हो तो मुझे बताइएगा। अलविदा।
B: अलविदा।
"""

# Split conversation into sentences
lines = [line.strip() for line in conversation_text.split("\n") if line.strip()]

# Generating summary, Keypoints and analysing sentiment of each sentence

In [15]:
# Sentiment analysis for each sentence
results = []
for line in lines:
    if ": " in line:
        speaker, message = line.split(": ", 1)
        sentiment = sentiment_pipeline(message)
        results.append({
            "speaker": speaker,
            "message": message,
            "sentiment": sentiment[0]['label'],
            "confidence": sentiment[0]['score']
        })

# Count sentiments for each speaker
ra_sentiments = {"positive": 0, "negative": 0, "neutral": 0}
b_sentiments = {"positive": 0, "negative": 0, "neutral": 0}

# Aggregate sentiment counts for each speaker
for res in results:
    if res["speaker"].startswith("RA"):
        ra_sentiments[res["sentiment"]] += 1
    elif res["speaker"].startswith("B"):
        b_sentiments[res["sentiment"]] += 1

# Determine overall sentiment based on majority
def determine_overall_sentiment(sentiment_counts):
    return max(sentiment_counts, key=sentiment_counts.get)

# Get overall sentiments for RA and B
overall_sentiment_ra = determine_overall_sentiment(ra_sentiments)
overall_sentiment_b = determine_overall_sentiment(b_sentiments)

# Generate Summary
summary_set = set()
for res in results:
    if res["speaker"].startswith("RA"):
        if "EMI" in res["message"] or "भुगतान" in res["message"]:
            summary_set.add("The recovery agent discussed the overdue EMI and emphasized timely payment.")
    if res["speaker"].startswith("B"):
        if "दिक्कत" in res["message"] or "नौकरी" in res["message"]:
            summary_set.add("The borrower mentioned facing financial difficulties due to job loss.")
        if "आधा भुगतान" in res["message"]:
            summary_set.add("The borrower proposed paying half the EMI and clearing the rest after finding a new job.")

# Convert set back to list for ordered output
summary = list(summary_set)

# Extract Key Actions
key_actions_set = set()
for res in results:
    if "अगले हफ्ते" in res["message"] or "SMS" in res["message"]:
        key_actions_set.add("The borrower agreed to pay half the EMI next week.")
    if "15 तारीख तक" in res["message"]:
        key_actions_set.add("The borrower agreed to pay the remaining EMI by the 15th of next month.")
    if "SMS भेज रहा हूं" in res["message"]:
        key_actions_set.add("The recovery agent will send SMS details for payment.")

# Convert set back to list for ordered output
key_actions = list(key_actions_set)

In [17]:
# Display Results

print("\nConcise Summary of the Conversation:\n")
for s in summary:
    print(f"- {s}")

print("\nKey Actions or Next Steps:\n")
for action in key_actions:
    print(f"- {action}")

print("\nSentence wise Sentiment Analysis:\n")
for res in results:
    print(f"{res['speaker']}: {res['message']} -> Sentiment: {res['sentiment']} (Confidence: {res['confidence']:.2f})")

print(f"\nOverall Sentiment for RA: {overall_sentiment_ra}")
print(f"Overall Sentiment for B: {overall_sentiment_b}")


Concise Summary of the Conversation:

- The borrower mentioned facing financial difficulties due to job loss.
- The recovery agent discussed the overdue EMI and emphasized timely payment.
- The borrower proposed paying half the EMI and clearing the rest after finding a new job.

Key Actions or Next Steps:

- The recovery agent will send SMS details for payment.
- The borrower agreed to pay half the EMI next week.
- The borrower agreed to pay the remaining EMI by the 15th of next month.

Sentence wise Sentiment Analysis:

Recovery Agent (RA): नमस्ते श्री कुमार, मैं एक्स वाई जेड फाइनेंस से बोल रहा हूं। आपके लोन के बारे में बात करनी थी। -> Sentiment: neutral (Confidence: 0.69)
Borrower (B): हां, बोलिए। क्या बात है? -> Sentiment: neutral (Confidence: 0.62)
RA: सर, आपका पिछले महीने का EMI अभी तक नहीं आया है। क्या कोई समस्या है? -> Sentiment: neutral (Confidence: 0.57)
B: हां, थोड़ी दिक्कत है। मेरी नौकरी चली गई है और मैं नया काम ढूंढ रहा हूं। -> Sentiment: negative (Confidence: 0.76)
RA: ओह